QUESTION 1 SOLUTION

In [2]:
import numpy as np
from typing import Tuple

D1 = 1.0  # Constant for d1

def inverse_kinematics(x: float, y: float, z: float) -> Tuple[float, float, float]:
    theta1 = np.arctan2(y, x)

    distance_xy = np.sqrt(x**2 + y**2)
    distance_z = z - D1

    theta2 = np.arctan2(distance_z, distance_xy) + np.pi/2
    d3 = np.sqrt(distance_xy**2 + distance_z**2)

    return theta1, theta2, d3

# Example usage
x, y, z = 1,2,2
state = inverse_kinematics(x, y, z)
print(state)


(1.1071487177940904, 1.9913306620788616, 2.4494897427831783)


QUESTION 2 SOLUTION

In [5]:
import numpy as np

def dh_transform_matrix(theta, d, a, alpha):
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    cos_alpha = np.cos(alpha)
    sin_alpha = np.sin(alpha)

    transformation_matrix = np.array([
        [cos_theta, -sin_theta * cos_alpha, sin_theta * sin_alpha, a * cos_theta],
        [sin_theta, cos_theta * cos_alpha, -cos_theta * sin_alpha, a * sin_theta],
        [0, sin_alpha, cos_alpha, d],
        [0, 0, 0, 1]
    ])

    return transformation_matrix

def end_effector_position(dh_params):
    num_joints = len(dh_params)
    end_effector_pose = np.identity(4)
    for i in range(num_joints):
        theta, d, a, alpha = dh_params[i]
        transformation_i = dh_transform_matrix(theta, d, a, alpha)
        end_effector_pose = np.dot(end_effector_pose, transformation_i)
    return end_effector_pose[:3, 3]

def manipulator_jacobian(dh_params, joint_types=None):
    if joint_types is None:
        joint_types = ['R'] * len(dh_params)

    if len(dh_params) != len(joint_types):
        raise ValueError("Number of DH parameters and joint types must be the same.")

    num_joints = len(dh_params)
    end_effector_pose = np.identity(4)
    jacobian = np.zeros((6, num_joints))

    for i in range(num_joints):
        theta, d, a, alpha = dh_params[i]
        transformation_i = dh_transform_matrix(theta, d, a, alpha)
        end_effector_pose = np.dot(end_effector_pose, transformation_i)

        if joint_types[i] == 'R':  # Revolute joint
            z_axis = end_effector_pose[:3, 2]
            position_difference = end_effector_position(dh_params) - end_effector_pose[:3, 3]
            jacobian[:3, i] = np.cross(z_axis, position_difference)
            jacobian[3:, i] = z_axis
        else:
            z_axis = end_effector_pose[:3, 2]
            jacobian[:3, i] = z_axis

    return jacobian

def joint_velocities(jacobian, end_effector_velocity):
    if np.size(jacobian, 1) == len(end_effector_velocity):
        joint_velocities = np.dot(np.linalg.pinv(jacobian), end_effector_velocity)
        return joint_velocities
    else:
        return "Error"

# Example usage: theta, d, a, alpha
dh_parameters = [
    [0, 1, 0, 0],
    [-np.pi/2, 1, 0, -np.pi/2],
    [-np.pi/2, 1, 0, -np.pi/2],
    [0, 1, 0, 0]
]

joint_types = ["P", "P", "P", "P"]
end_effector_velocity = [1, 1, 1, 1, 1, 1]

jacobian = manipulator_jacobian(dh_parameters, joint_types)
end_effector_pos = end_effector_position(dh_parameters)
joint_velocities_result = joint_velocities(jacobian, end_effector_velocity)

print("Manipulator Jacobian:")
print(jacobian)
print("End-effector Position:")
print(end_effector_pos)
print("End-effector Velocity:")
print(joint_velocities_result)


Manipulator Jacobian:
[[ 0.0000000e+00  1.0000000e+00  1.2246468e-16  1.2246468e-16]
 [ 0.0000000e+00  6.1232340e-17 -1.0000000e+00 -1.0000000e+00]
 [ 1.0000000e+00  6.1232340e-17 -6.1232340e-17 -6.1232340e-17]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00]]
End-effector Position:
[ 1. -1.  2.]
End-effector Velocity:
Error


QUESTION 5 SOLUTION

In [6]:
import numpy as np

def spherical_wrist_inverse_kinematics(r36):
    if r36[0, 2] != 0 or r36[1, 2] != 0:
        theta_5 = np.arctan2(-np.sqrt(1 - r36[2, 2]**2), r36[2, 2])
        theta_4 = np.arctan2(-r36[1, 2], -r36[0, 2])
        theta_6 = np.arctan2(-r36[2, 1], r36[2, 0])
    else:
        theta_4 = 0.
        if r36[2, 2] > 0:
            theta_5 = 0
            theta_6 = 0  # Considering
            theta_4 = np.arctan2(r36[1, 0], r36[0, 0])
        else:
            theta_5 = np.pi
            theta_6 = 0  # Considering
            theta_4 = -np.arctan2(-r36[0, 1], -r36[0, 0])

    return np.array([theta_4, theta_5, theta_6])

# Example usage:
# Assuming r36 is the rotation matrix
r36_example = np.array([[0.1, 0.2, 0.3],
                        [0.4, 0.5, 0.6],
                        [0.7, 0.8, 0.9]])

joint_angles = spherical_wrist_inverse_kinematics(r36_example)
print("Spherical Wrist Joint Angles:")
print(joint_angles)


Spherical Wrist Joint Angles:
[-2.03444394 -0.45102681 -0.85196633]
